第十四章  可迭代的对象、迭代器和生成器

扫描内存中放不下的数据集时，我们要找到一种惰性获取数据项的方式，即按需一次获取一个数据项。这就是迭代器模式。

所有生成器都是迭代器，因为生成器完全实现了迭代器的接口。在python社区中，大多数时候把迭代器和生成器视作一个概念。

14.1 Sentence类第1版：单词序列

In [45]:
#senten.py 把句子划分为单词序列
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)
    
    def __getitem__(self, position):
        return self.words[position]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)

In [22]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [23]:
iter(s)

In [24]:
s.__dict__

{'text': '"The time has come," the Walrus said',
 'words': ['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']}

In [5]:
for word in s:#Sentence实例可以迭代！
    print(word)

The
time
has
come
the
Walrus
said


In [6]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [7]:
s[0]

'The'

In [8]:
s[-1]

'said'

![chapter14-2](image/chapter14-2.jpg)

14.2 可迭代对象与迭代器的对比

可迭代对象：使用iter内置函数可以获取迭代器的对象。如果对象实现了能返回迭代器的__iter__方法，那么对象是可迭代的。序列都可以迭代；实现了__getitem__方法，而且其参数是从0开始的索引，这种对象也可迭代。

明确可迭代对象和迭代器之间的关系：python从可迭代对象中获取迭代器。

In [9]:
s='ABC'
for char in s:
    print(char)

A
B
C


In [10]:
s='ABC'
it=iter(s)#使用可迭代对象构建迭代器it
while True:
    try:print(next(it))#不断在迭代器上调用next函数，获取下一个字符
    except StopIteration:#没字符了，抛出异常
        del it#释放对it的引用，即废弃迭代器对象
        break

A
B
C


标准的迭代器接口有两个方法：

__next__

__iter__返回self，实例本身，以便在需要使用可迭代对象的地方使用迭代器，例如for循环

![chapter14-4](image/chapter14-4.jpg)

In [11]:
s3=Sentence('Pig and Pepper')
it=iter(s3)
it

In [12]:
next(it)

'Pig'

In [13]:
next(it)

'and'

In [14]:
next(it)

'Pepper'

In [15]:
next(it) #迭代器没有单词了，抛出StopIteration异常

StopIteration: 

In [17]:
list(it) #到头后，迭代器没用了

[]

In [18]:
list(iter(s3)) #想再次迭代，需重构迭代器

['Pig', 'and', 'Pepper']

14.3 Sentence类第2版：典型的迭代器

In [25]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)
    
    def __iter__(self):#与前一版相比，这里只多了一个__iter__方法。这里没__getitem方法，为的是明确表明这个类可以迭代是因为实现了__iter__方法
        return SentenceIterator(self.words) #返回一个SentenceIterator实例

#构建SentenceIterator
class SentenceIterator:
    def __init__(self,words):
        self.words=words
        self.index=0
    
    def __next__(self):
        try:
            word=self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index+=1
        return word
    
    def __iter__(self):
        return self

In [26]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [27]:
for char in s:
    print(char)

The
time
has
come
the
Walrus
said


In [29]:
s.__dict__

{'text': '"The time has come," the Walrus said',
 'words': ['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']}

In [31]:
iter(s)
    

In [34]:
it=iter(s)
it2=iter(it)
it,it2

(<__main__.SentenceIterator at 0x1bcaf356160>,
 <__main__.SentenceIterator at 0x1bcaf356160>)

把Sentence变成迭代器：坏主意

可迭代对象有个__iter__方法，每次都可实例化一个新的迭代器；而迭代器要实现__next__方法，返回单个元素；此外还要实现__iter__方法，返回迭代器本身。

除了__iter__方法除外，你可能还想再Sentence类实现__next__,让Sentence实例既是可迭代对象又是自身的迭代器，但这种想法十分糟糕。

注意，迭代器模式可用来：

(1).访问一个聚合对象的内容而无需暴露它的内部表示

(2).支持对聚合对象多种遍历

(3).为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的迭代器，而且各个迭代器要能维护自身的内部状态，因此这一模式的正确实现方式是，每次调用iter(my iterable)新建一个独立的迭代器。这就是为什么这个示例需要定义SentenceIterator类。


可迭代对象一定不能是自身的迭代器。也就是说，可迭代对象必须实现__iter__方法，但不能实现__next__方法。
另一方面，迭代器应该一直可以迭代。迭代器的__iter__方法应该返回自身。


关于迭代器为什么一定要有__iter__方法，简单的说，如果迭代器不实现_iter_方法的话，那么迭代器本身就不能迭代了，不是可迭代对象。

比如下面这个例子，自己定义的SentenceIterator如果不实现_iter_方法的话，就不可迭代了，违反了迭代器一定是可迭代对象的约定

In [49]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)
    
    def __iter__(self):#与前一版相比，这里只多了一个__iter__方法。这里没__getitem方法，为的是明确表明这个类可以迭代是因为实现了__iter__方法
        return SentenceIterator(self.words) #返回一个SentenceIterator实例

#构建SentenceIterator
class SentenceIterator:
    def __init__(self,words):
        self.words=words
        self.index=0
    
    def __next__(self):
        try:
            word=self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index+=1
        return word
    

In [50]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [51]:
iter(s)

In [52]:
it=iter(s)
for i in it:
    print(i)

TypeError: 'SentenceIterator' object is not iterable

14.4 Sentence类第三版：生成器函数

实现相同功能，但却符合python习惯的方式是，用生成器函数代替SentenceIterator类。

In [53]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:
    def __init__(self,text):
        self.text=text
        self.words=RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)'%reprlib.repr(self.text)
    
    #这里的__iter__方法是生成器函数
    def __iter__(self):
        for word in self.words:#迭代self.words
            yield word#产出当前word
        return #return这个语句不是必要的，这个函数可以直接落空，自动返回。不管有没有return语句，
               # 生成器函数都不会抛出StopIteration，而是在生成完全部值后直接退出

In [54]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [55]:
for i in s:
    print(i)

The
time
has
come
the
Walrus
said


In [57]:
it=iter(s)
it

<generator object __iter__ at 0x000001BCAF280FC0>

In [58]:
for i in it:
    print(i)

The
time
has
come
the
Walrus
said


只要python函数中包含关键字yield，该函数就是生成器函数。调用生成器函数返回一个生成器对象。

生成器函数的定义体中一般有循环，但这不是必要条件如下例：

In [59]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [60]:
gen_123

<function __main__.gen_123>

In [61]:
gen_123()

<generator object gen_123 at 0x000001BCAF342B48>

In [62]:
for i in gen_123():#生成器是迭代器，会生成传给yield关键字的表达式的值
    print(i)

1
2
3


In [68]:
g=gen_123()#g是迭代器
next(g)

1

In [64]:
next(g)

2

In [65]:
next(g)

3

In [66]:
next(g)

StopIteration: 

In [70]:
g=gen_123()#g是迭代器
next(g)
for i in g:
    print(i)

2
3


把生成器传给next(...)时，生成器函数会向前，执行函数定义体下一个yield语句，返回产出的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出StopIteration异常——这一点与迭代器协议一致。

In [71]:
def gen_AB():
    print('start')
    yield  'A'
    print('continue')
    yield 'B'
    print('end')
    yield 'C'
for c in gen_AB():#迭代时，for机制的作用与g=iter(gen_AB())一样，用于获取生成器对象，然后每次迭代时隐式调用next(g)
    print('-->',c)

start
--> A
continue
--> B
end
--> C


In [72]:
next(gen_AB())

start


'A'

14.5 Sentence类第四版：惰性实现

目前实现的几版Sentence类都不具有惰性，因为__init__方法急迫的构建了文本中的单词列表，然后将其绑定到self.words上，并不节省内存。如果只需迭代前几个单词，大多数工作是白费力气。

re.finditer函数是re.findall函数的惰性版本，返回的不是一个列表，而是一个生成器，按需生产re.MatchObject实例。节省内存

In [7]:
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  #不再需words列表

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text): #finditer函数构建一个迭代器，包含self.texxt中匹配RE_WORD的单词，产出MatchObject对象
            yield match.group()

In [8]:
s=Sentence('"The time has come," the Walrus said')
s

Sentence('"The time ha...e Walrus said')

In [9]:
for i in s:#迭代时，for机制的作用与g=iter(s)一样，用于获取生成器对象，然后每次迭代时隐式调用next(g)
    print(i)

The
time
has
come
the
Walrus
said


In [83]:
it=iter(s)
it

<generator object __iter__ at 0x000001BCAF342E08>

In [84]:
next(it)

'The'

14.6 Sentence第五版：生成器表达式

简单的生成器函数，可以替换为生成器表达式

生成器表达式可以理解为列表推导的惰性版本：不会迫切的构建列表，而是返回一个生成器，按需惰性生成元素。

In [10]:
def gen_AB():
    print('start')
    yield  'A'
    print('continue')
    yield 'B'
    print('end')
    yield 'C'

In [11]:
res1=[x*3 for x in gen_AB()]#列表推导迫切的迭代gen_AB()函数生成的生成器对象产出的元素：A,B,C。注意，下面的输出。

start
continue
end


In [12]:
res1

['AAA', 'BBB', 'CCC']

In [87]:
for i in res1:#这个循环迭代列表推导生成的res1列表
    print('-->',i)

--> AAA
--> BBB
--> CCC


In [89]:
res2=(x*3 for x in gen_AB())#一个生成器对象

In [90]:
res2

<generator object <genexpr> at 0x000001BCAF360410>

In [91]:
for i in res2:
    print('-->',i)

start
--> AAA
continue
--> BBB
end
--> CCC


In [94]:
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  #不再需words列表

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

In [95]:
s=Sentence('"The time has come," the Walrus said')
s
for i in s:
    print(i)

The
time
has
come
the
Walrus
said


14.8 另一个示例：等差数列生成器

In [13]:
class ArithmeticProgression:
    def __init__(self,begin,step,end=None):
        self.begin=begin
        self.step=step
        self.end=end
    
    def __iter__(self):
        result=type(self.begin+self.step)(self.begin)#把self.begin的类型转换成type(self.begin+self.step)
        forever = self.end is None
        index=0
        while forever or result <self.end:
            yield  result
            index+=1
            result=self.begin+self.step*index
            

In [14]:
ap=ArithmeticProgression(0,1,3)

In [15]:
list(ap)

[0, 1, 2]

In [16]:
ap=ArithmeticProgression(1,0.5,3)

In [17]:
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [18]:
from fractions import Fraction
ap=ArithmeticProgression(0,Fraction(1,3),1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [19]:
ap

In [20]:
from decimal import Decimal
ap=ArithmeticProgression(0,Decimal('.1'),.3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [23]:
#代码量更少的生成器函数
def aritprog_gen(begin,step,end=None):
    result=type(begin+step)(begin)
    forever=end is None
    index=0
    while forever or result<end:
        yield  result
        index+=1
        result=begin+step*index

In [24]:
#始终记住，标准库中有很多现成的生成器
#使用itertools模块生成等差数列
import itertools
gen = itertools.count(1,.5)#如果不传入参数，itertools.count函数会生成从零开始的整数数列
next(gen)

1

In [25]:
next(gen)

1.5

In [27]:
#itertools.count函数从不停止，因此如果调用list(count())，python会创建一个特别大的列表，超出可用内存，在调用失败之前，电脑疯狂运转
#不过，itertools.takewhile不同，它会生成一个使用另一个生成器的生成器，在指定的条件计算结果为false时停止。
# 因此可把这两个函数结合在一起使用。
gen=itertools.takewhile(lambda n:n<3,itertools.count(1,.5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [ ]:
import itertools
def aritprog_gen(begin,step,end=None):
    first=type(begin+step)(begin)
    ap_gen=itertools.count(begin,step)
    if end is not None:
        ap_gen=itertools.takewhile(lambda n:n<end,ap_gen)
    return ap_gen

14.9 标准库中的生成器函数

![chapter14-9](image/chapter14-9.jpg)

In [29]:
def vowel(c):
    return c.lower() in 'aeiou'
list(filter(vowel,'Aardvark'))

['A', 'a', 'a']

In [30]:
list(itertools.filterfalse(vowel,'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [31]:
gen=itertools.dropwhile(vowel,'Aardvark')
gen

In [32]:
list(gen)

['r', 'd', 'v', 'a', 'r', 'k']

In [33]:
list(itertools.takewhile(vowel,'Aardvark'))

['A', 'a']

In [35]:
list(itertools.compress('Aardvark',(1,0,1,1,0,1)))

['A', 'r', 'd', 'a']

In [36]:
list(itertools.islice('Aardvark',4))

['A', 'a', 'r', 'd']

In [37]:
list(itertools.islice('Aardvark',4,7))

['v', 'a', 'r']

In [38]:
list(itertools.islice('Aardvark',1,7,2))

['a', 'd', 'a']

![chapter14-9-2](image/chapter14-9-2.jpg)

In [39]:
sample=[5,4,2,8,7,6,3,0,9,1]
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [40]:
list(itertools.accumulate(sample,min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [41]:
list(itertools.accumulate(sample,max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [42]:
import operator
list(itertools.accumulate(sample,operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [43]:
list(enumerate('albatroz',1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [44]:
import operator
list(map(operator.mul,range(11),range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [45]:
list(map(lambda a,b:(a,b),range(11),[2,4,8]))#作用等同于zip

[(0, 2), (1, 4), (2, 8)]

In [46]:
list(itertools.starmap(operator.mul,enumerate('albatroz',1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [47]:
list(itertools.starmap(lambda a,b:b/a,
                       enumerate(itertools.accumulate(sample),1)))#计算平均值

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

![chapter14-9-3](image/chapter-9-3.jpg)

In [48]:
list(itertools.chain('ABC',range(2)))

['A', 'B', 'C', 0, 1]

In [50]:
list(itertools.chain(enumerate('ABC')))#如果只传入一个可迭代对象，那么chain函数没什么用

[(0, 'A'), (1, 'B'), (2, 'C')]

In [51]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [52]:
list(zip('ABC',range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [53]:
list(zip('ABC',range(5),[10,20,30,40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [54]:
list(itertools.zip_longest('ABC',range(5),[10,20,30,40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30), (None, 3, 40), (None, 4, None)]

In [55]:
list(itertools.zip_longest('ABC',range(5),[10,20,30,40],fillvalue='?'))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30), ('?', 3, 40), ('?', 4, '?')]

In [57]:
list(itertools.product('ABC',range(2)))#计算笛卡尔积

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [58]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [59]:
list(itertools.product('ABC',repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [61]:
list(itertools.product(range(2),repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [62]:
list(itertools.product('AB',range(2),repeat=2))

[('A', 0, 'A', 0),
 ('A', 0, 'A', 1),
 ('A', 0, 'B', 0),
 ('A', 0, 'B', 1),
 ('A', 1, 'A', 0),
 ('A', 1, 'A', 1),
 ('A', 1, 'B', 0),
 ('A', 1, 'B', 1),
 ('B', 0, 'A', 0),
 ('B', 0, 'A', 1),
 ('B', 0, 'B', 0),
 ('B', 0, 'B', 1),
 ('B', 1, 'A', 0),
 ('B', 1, 'A', 1),
 ('B', 1, 'B', 0),
 ('B', 1, 'B', 1)]

![chapter14-9-4](image/chapter14-9-4.jpg)

In [63]:
ct=itertools.count()
next(ct),next(ct),next(ct)

(0, 1, 2)

In [64]:
list(itertools.islice(itertools.count(1,.3),4))

[1, 1.3, 1.6, 1.9000000000000001]

In [65]:
cy=itertools.cycle('ABC')
next(cy)

'A'

In [66]:
list(itertools.islice(cy,7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [67]:
rp=itertools.repeat(7)
next(rp),next(rp)

(7, 7)

In [68]:
list(itertools.repeat(8,4))

[8, 8, 8, 8]

In [69]:
list(map(operator.mul,range(11),itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [71]:
#组合学生成器
list(itertools.combinations('ABC',2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [72]:
list(itertools.combinations_with_replacement('ABC',2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [73]:
list(itertools.permutations('ABC',2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [75]:
list(itertools.product('ABC',repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

![chapter14-9-5](image/chapter14-9-5.jpg)

In [77]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x239bf51e668>),
 ('A', <itertools._grouper at 0x239bf51e9b0>),
 ('G', <itertools._grouper at 0x239bf529048>)]

In [78]:
for char,group in itertools.groupby('LLLLAAGGG'):
    print(char,'->',list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G', 'G']


In [79]:
animal=['duck','eagle','rat','giraffe','bear','bat','dolphin','lion']
animal.sort(key=len)

In [80]:
animal

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'giraffe', 'dolphin']

In [81]:
for length,group in itertools.groupby(animal,len):
    print(length,'->',list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle']
7 -> ['giraffe', 'dolphin']


In [82]:
for length,group in itertools.groupby(reversed(animal),len):
    print(length,'->',list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [83]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x239bf52d508>, <itertools._tee at 0x239bf52d548>]

In [83]:
g1,g2=itertools.tee('ABC')

In [85]:
next(g1)

'A'

In [86]:
next(g2)

'A'

In [87]:
next(g2)

'B'

In [88]:
list(g1)

['B', 'C']

In [89]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

14.10 yield from

In [90]:
def chain(*iterable):
    for it in iterable:
        for i in it:
            yield i
s='ABC'
t=tuple(range(3))
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

In [91]:
def chain(*iterable):
    for it in iterable:
        yield from it#替代了内层的for循环

In [92]:
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

14.11可迭代的归约函数

![chapter14-11](image/chapter14-11.jpg)

其实，这里列出的每个内置函数都可以用functools.reduce函数实现。对all和any来说，有一项重要的优化措施是reduc做不到的：函数会短路（即确定了结果就立即停止使用迭代器）

In [93]:
all([1,2,3])

True

In [94]:
all([1,0,3])

False

In [95]:
all([])

True

In [96]:
any([0,0.0])

False

In [97]:
any([0,0.1])

True

14.12深入分析iter函数

In [98]:
from random import randint
def d6():
    return randint(1,6)

d6_iter=iter(d6,1)#1为哨符，当可调用对象返回这个值时，触发迭代器抛出StopInteration异常，而不产出哨符

In [101]:
d6_iter

In [102]:
for roll in d6_iter:
    print(roll)

6
6
3
